# EMA-Strategy Interactive Brokers execution

In [1]:
import ibapi
import pandas as pd


print(pd.__version__)
print(ibapi.__version__)


1.2.4
9.81.1


In [2]:
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import *

import pandas as pd
import threading
import time
import datetime


class IBapi(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.bardata = {} #Initialize dictionary to store bar data

    def nextValidId(self, orderId: int):
        super().nextValidId(orderId)
        self.nextorderId = orderId
        print('The next valid order id is: ', self.nextorderId)

    def tick_df(self, reqId, contract):
        ''' custom function to init DataFrame and request Tick Data '''
        self.bardata[reqId] = pd.DataFrame(columns=['time', 'price'])
        self.bardata[reqId].set_index('time', inplace=True)
        self.reqTickByTickData(reqId, contract, "Last", 0, True)
        return self.bardata[reqId]

    def tickByTickAllLast(self, reqId, tickType, time, price, size, tickAtrribLast, exchange, specialConditions):
        if tickType == 1:
            self.bardata[reqId].loc[pd.to_datetime(time, unit='s')] = price

    def futures_contract(self, symbol, secType='FUT', exchange='CME'):
        ''' custom function to create contract '''
        contract = Contract()
        contract.symbol = symbol
        contract.secType = secType
        contract.exchange = exchange
        contract.lastTradeDateOrContractMonth = "202303"
        contract.multiplier = 20
        return contract


def run_loop():
    app.run()

def submit_order(contract, direction, inv_direction, stp_price, tp_price, qty=1):
    #Create order object
    order = Order()
    order.action = direction
    order.totalQuantity = qty
    order.orderType = 'MKT'
    order.eTradeOnly = ''
    order.firmQuoteOnly = ''
    order.orderId = app.nextorderId
    app.nextorderId += 1
    order.eTradeOnly = ''
    order.firmQuoteOnly = ''
    order.transmit = False
    
    stop_order = Order()
    stop_order.action = inv_direction
    stop_order.totalQuantity = qty
    stop_order.orderType = 'STP'
    stop_order.auxPrice = stp_price
    stop_order.orderId = app.nextorderId
    app.nextorderId += 1
    stop_order.parentId = order.orderId
    stop_order.eTradeOnly = ''
    stop_order.firmQuoteOnly = ''
    stop_order.transmit = False
    
    takeprofit_order = Order()
    takeprofit_order.action = inv_direction
    takeprofit_order.totalQuantity = qty
    takeprofit_order.orderType = 'LMT'
    takeprofit_order.lmtPrice = tp_price
    takeprofit_order.orderId = app.nextorderId
    app.nextorderId += 1
    takeprofit_order.parentId = order.orderId
    takeprofit_order.eTradeOnly = ''
    takeprofit_order.firmQuoteOnly = ''
    takeprofit_order.transmit = True
  
    #submit order
    time.sleep(1)
    print('Placing {} order'.format(direction))
    app.placeOrder(order.orderId, contract, order)
    time.sleep(1)
    print('Placing STOP-LOSS order')
    app.placeOrder(stop_order.orderId, contract, stop_order)
    time.sleep(1)
    print('Placing TAKE-PROFIT order')
    app.placeOrder(takeprofit_order.orderId, contract, takeprofit_order)
    app.nextorderId += 1
    
    
    print('Full bracket order placed')
    print('Order placed at time: ' + str(datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")))
    print('Sleep Time for Next Signal (300s)')
    time.sleep(300)
    print('Sleep Time Done')
    

def check_for_trade(df, contract):

    start_time = df.index[-1] - pd.Timedelta(minutes=3)
    df.index = pd.to_datetime(df.index, unit='m')
    
    data_open = df.resample('min').first()
    data_close = df.resample('min').last()
    data_high = df.resample('min').max()
    data_low = df.resample('min').min()
    
    data = df.copy()
    
    data = data.resample('min').mean()
    data['Open'] = data_open
    data['Close'] = data_close
    data['High'] = data_high
    data['Low'] = data_low

    
    trend_lag  = 3
    

    data['ema-short'] = data['Close'].shift(1).ewm(alpha=(2/(9+1))).mean()
    data['ema-short-trend'] = data['ema-short'] - data['ema-short'].shift(trend_lag)

    data['ema-mid'] = data['Close'].shift(1).ewm(alpha=(2/(50+1))).mean()
    data['ema-mid-trend'] = data['ema-mid'] - data['ema-mid'].shift(trend_lag)
    data['lower_threshold'] = data['ema-mid'] * 0.99995
    data['higher_threshold'] = data['ema-mid'] * 1.00015

    data['ema-long'] = data['Close'].shift(1).ewm(alpha=(2/(100+1))).mean()
    data['ema-long-trend'] = data['ema-long'] - data['ema-long'].shift(trend_lag)

    def quarter_round(x):
        return round(x*4)/4
    
    # Buy signal
    if (data['ema-short-trend'][-1] > 0) and (data['ema-mid-trend'][-1] > 0) and (data['ema-long-trend'][-1] > 0):
        
        # Once trend is confirmed, evaluate if the long and short moving average have the percentage difference 
        # from the middle averge.
        # For bullish: short MA should be above higher threshold, long MA below the lower threshold.
        #if (data['ema-short'][-1] > data['higher_threshold'][i]) and (data['ema-long'][-1] < data['lower_threshold'][i]):
            


        if (data['Close'][-3] < data['ema-short'][-3]) and (data['Close'][-2] > data['ema-short'][-2]):
                
            stp_p = (data['Open'][-2] + data['Close'][-2])/2
            stp_p = quarter_round(stp_p)
            
            take_p = abs(data['High'][-2] - data['Open'][-2])
            tp_p = data['Close'][-2] + take_p
            tp_p = quarter_round(tp_p)

            print('Signal: Buy')
            print('Stop Loss: ' + str(stp_p))
            print('Take Profit: ' + str(tp_p))

            submit_order(contract, direction='BUY', inv_direction='SELL', stp_price=stp_p, tp_price=tp_p)
            return True


    # Bearish rules
    elif (data['ema-short-trend'][-1] < 0) and (data['ema-mid-trend'][-1] < 0) and (data['ema-long-trend'][-1] < 0):


        # For bearish: short MA should be below lower threshold, long MA above the higher threshold
        #if (data['ema-short'][-1] < data['lower_threshold'][-1]) and (data['ema-long'][-1] > data['higher_threshold'][-1]):
            


        if (data['Close'][-3] > data['ema-short'][-3]) and (data['Close'][-2] < data['ema-short'][-2]):
                
            stp_p = (data['Close'][-2] + data['Open'][-2])/2
            stp_p = quarter_round(stp_p)
            
            take_p = abs(data['Low'][-2] - data['Open'][-2])
            tp_p = data['Close'][-2] - take_p
            tp_p = quarter_round(tp_p)
            print('Signal: Sell')
            print('Stop Loss: ' + str(stp_p))
            print('Take Profit: ' + str(tp_p))
            submit_order(contract, direction='SELL', inv_direction='BUY', stp_price=stp_p, tp_price=tp_p)
            return True


#Main
app = IBapi()
app.nextorderId = None
app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop)
api_thread.start()

#Check if the API is connected via orderid
while True:
    if isinstance(app.nextorderId, int):
        print('Connected')
        break
    else:
        print('Waiting for connection')
        time.sleep(1)

#Create contract object
NQ_contract = app.futures_contract('NQ')

#-----------------------------------------------------------#

#Request tick data for google using custom function
df = app.tick_df(401, NQ_contract)

print('Requesting tick data used for order conditions calculations')

#Verify data stream
time.sleep(10)
for i in range(100):
    if len(df) > 0:
        break
    
    time.sleep(0.3)

if i == 99:
    app.disconnect()
    raise Exception ('Error with Tick data stream')

print('Streaming tick data')

#Check if there is enough data
data_length = df.index[-1] - df.index[0]
if data_length.seconds < 180:
    time.sleep(180 - data_length.seconds)
    
print('Tick data with enough size to calculate EMAs')
print('Waiting for 3-EMA signal')
#Main loop
while True:
    check_for_trade(df, NQ_contract)
    time.sleep(1)

app.disconnect()

Waiting for connection
The next valid order id is:  434


ERROR -1 2104 Market data farm connection is OK:hfarm
ERROR -1 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 2104 Market data farm connection is OK:jfarm
ERROR -1 2104 Market data farm connection is OK:usfuture
ERROR -1 2104 Market data farm connection is OK:cashfarm
ERROR -1 2104 Market data farm connection is OK:eufarmnj
ERROR -1 2104 Market data farm connection is OK:usfarm
ERROR -1 2106 HMDS data farm connection is OK:euhmds
ERROR -1 2106 HMDS data farm connection is OK:ushmds.nj
ERROR -1 2106 HMDS data farm connection is OK:cashhmds
ERROR -1 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 2106 HMDS data farm connection is OK:ushmds
ERROR -1 2158 Sec-def data farm connection is OK:secdefil


Connected
Requesting tick data used for order conditions calculations
Streaming tick data


ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.hfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.jfarm
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj
ERROR -1 2108 Market data farm connection is inactive but should be available upon demand.eufarmnj


Tick data with enough size to calculate EMAs
Waiting for 3-EMA signal
Signal: Buy
Stop Loss: 11403.5
Take Profit: 11406.75
Placing BUY order
Placing STOP-LOSS order
Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-19 23:52:04
Sleep Time for Next Signal (300s)


...

ERROR 463 399 Order Message: SELL 1 NQ MAR'23 Warning: your order will not be placed at the exchange until 2023-01-20 08:30:00 US/Central


Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-20 01:39:07
Sleep Time for Next Signal (300s)
Sleep Time Done
Signal: Buy
Stop Loss: 11424.0
Take Profit: 11427.0
Placing BUY order
Placing STOP-LOSS order
Placing TAKE-PROFIT order
Full bracket order placed
Order placed at time: 2023-01-20 01:44:11
Sleep Time for Next Signal (300s)


ERROR 467 399 Order Message: SELL 1 NQ MAR'23 Warning: your order will not be placed at the exchange until 2023-01-20 08:30:00 US/Central
ERROR 466 399 Order Message: BUY 1 NQ MAR'23 Warning: Your order was repriced so as not to cross a related resting order.
ERROR 463 202 Order Canceled - reason:
ERROR 463 202 Order Canceled - reason:
ERROR 467 202 Order Canceled - reason:
ERROR 467 202 Order Canceled - reason:


Sleep Time Done


ValueError: Length mismatch: Expected axis has 3751 elements, new values have 3750 elements

- New strat
- Optimization
- Storage structure
- Fix when error happens rerun code